In [1]:
import math
import os

import numpy as np
import torch

from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.sampler import Sampler

In [2]:

def read_file(_path, delim='\t'):
    data = []
    if delim == 'tab':
        delim = '\t'
    elif delim == 'space':
        delim = ' '
    with open(_path, 'r') as f:
        for line in f:
            line = line.strip().split(delim)
            line = [float(i) for i in line]
            data.append(line)
    return np.asarray(data)

def poly_fit(traj, traj_len, threshold):
    """
    Input:
    - traj: Numpy array of shape (2, traj_len)
    - traj_len: Len of trajectory
    - threshold: Minimum error to be considered for non-linear traj
    Output:
    - int: 1 -> Non Linear 0-> Linear
    """
    t = np.linspace(0, traj_len - 1, traj_len)
    res_x = np.polyfit(t, traj[0, -traj_len:], 2, full=True)[1]
    res_y = np.polyfit(t, traj[1, -traj_len:], 2, full=True)[1]
    if res_x + res_y >= threshold:
        return 1.0
    else:
        return 0.0

class TrajectoryDataset(Dataset):
    """Dataloder for the Trajectory datasets"""

    def __init__(self, data_dir, obs_len=8, pred_len=12, skip=1, threshold=0.02, min_ped=1, delim='\t'):
        """
        Args:
        - data_dir: Directory containing dataset files in the format <frame_id> <ped_id> <x> <y>
        - obs_len: Number of time-steps in input trajectories
        - pred_len: Number of time-steps in output trajectories
        - skip: Number of frames to skip while making the dataset
        - threshold: Minimum error to be considered for non-linear traj when using a linear predictor
        - min_ped: Minimum number of pedestrians that should be in a sequence
        - delim: Delimiter in the dataset files
        """
        super(TrajectoryDataset, self).__init__()

        self.data_dir = data_dir
        self.obs_len = obs_len
        self.pred_len = pred_len
        self.skip = skip
        self.seq_len = self.obs_len + self.pred_len
        self.delim = delim

        all_files = os.listdir(self.data_dir)
        all_files = [os.path.join(self.data_dir, _path) for _path in all_files]
        num_peds_in_seq = []
        seq_list = []
        loss_mask_list = []
        non_linear_ped = []
        for path in all_files:
            data = read_file(path, delim)
            frames = np.unique(data[:, 0]).tolist()
            frame_data = []
            for frame in frames:
                frame_data.append(data[frame == data[:, 0], :])
            num_sequences = int(math.ceil((len(frames) - self.seq_len + 1) / skip))

            for idx in range(0, num_sequences * self.skip + 1, skip):
                curr_seq_data = np.concatenate(frame_data[idx:idx + self.seq_len], axis=0)
                peds_in_curr_seq = np.unique(curr_seq_data[:, 1])
                curr_seq = np.zeros((len(peds_in_curr_seq), 2, self.seq_len))
                curr_loss_mask = np.zeros((len(peds_in_curr_seq), self.seq_len))
                num_peds_considered = 0
                _non_linear_ped = []
                for _, ped_id in enumerate(peds_in_curr_seq):
                    curr_ped_seq = curr_seq_data[curr_seq_data[:, 1] == ped_id, :]
                    curr_ped_seq = np.around(curr_ped_seq, decimals=4)
                    pad_front = frames.index(curr_ped_seq[0, 0]) - idx
                    pad_end = frames.index(curr_ped_seq[-1, 0]) - idx + 1
                    if pad_end - pad_front != self.seq_len:
                        continue
                    curr_ped_seq = np.transpose(curr_ped_seq[:, 2:])
                    curr_ped_seq = curr_ped_seq
                    _idx = num_peds_considered
                    curr_seq[_idx, :, pad_front:pad_end] = curr_ped_seq
                    # Linear vs Non-Linear Trajectory
                    _non_linear_ped.append(poly_fit(curr_ped_seq, pred_len, threshold))
                    curr_loss_mask[_idx, pad_front:pad_end] = 1
                    num_peds_considered += 1

                if num_peds_considered > min_ped:
                    non_linear_ped += _non_linear_ped
                    num_peds_in_seq.append(num_peds_considered)
                    loss_mask_list.append(curr_loss_mask[:num_peds_considered])
                    seq_list.append(curr_seq[:num_peds_considered])

        self.num_seq = len(seq_list)
        seq_list = np.concatenate(seq_list, axis=0)
        loss_mask_list = np.concatenate(loss_mask_list, axis=0)
        non_linear_ped = np.asarray(non_linear_ped)
        self.num_peds_in_seq = np.array(num_peds_in_seq)

        # Convert numpy -> Torch Tensor
        self.obs_traj = torch.from_numpy(seq_list[:, :, :self.obs_len]).type(torch.float).permute(0, 2, 1)  # NTC
        self.pred_traj = torch.from_numpy(seq_list[:, :, self.obs_len:]).type(torch.float).permute(0, 2, 1)  # NTC
        self.loss_mask = torch.from_numpy(loss_mask_list).type(torch.float)
        self.non_linear_ped = torch.from_numpy(non_linear_ped).type(torch.float)
        cum_start_idx = [0] + np.cumsum(num_peds_in_seq).tolist()
        self.seq_start_end = [(start, end) for start, end in zip(cum_start_idx, cum_start_idx[1:])]

    def __len__(self):
        return self.num_seq

    def __getitem__(self, index):
        start, end = self.seq_start_end[index]
        out = [self.obs_traj[start:end], self.pred_traj[start:end],
               self.non_linear_ped[start:end], self.loss_mask[start:end], None, [[0, end - start]]]
        return out


class TrajBatchSampler(Sampler):
    r"""Samples batched elements by yielding a mini-batch of indices.
    Args:
        data_source (Dataset): dataset to sample from
        batch_size (int): Size of mini-batch.
        shuffle (bool, optional): set to ``True`` to have the data reshuffled
            at every epoch (default: ``False``).
        drop_last (bool): If ``True``, the sampler will drop the last batch if
            its size would be less than ``batch_size``
        generator (Generator): Generator used in sampling.
    """

    def __init__(self, data_source, batch_size=64, shuffle=False, drop_last=False, generator=None):
        self.data_source = data_source
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.drop_last = drop_last
        self.generator = generator

    def __iter__(self):
        assert len(self.data_source) == len(self.data_source.num_peds_in_seq)

        if self.shuffle:
            if self.generator is None:
                generator = torch.Generator()
                generator.manual_seed(int(torch.empty((), dtype=torch.int64).random_().item()))
            else:
                generator = self.generator
            indices = torch.randperm(len(self.data_source), generator=generator).tolist()
        else:
            indices = list(range(len(self.data_source)))
        num_peds_indices = self.data_source.num_peds_in_seq[indices]

        batch = []
        total_num_peds = 0
        for idx, num_peds in zip(indices, num_peds_indices):
            batch.append(idx)
            total_num_peds += num_peds
            if total_num_peds >= self.batch_size:
                yield batch
                batch = []
                total_num_peds = 0
        if len(batch) > 0 and not self.drop_last:
            yield batch

    def __len__(self):
        # Approximated number of batches.
        # The order of trajectories can be shuffled, so this number can vary from run to run.
        if self.drop_last:
            return sum(self.data_source.num_peds_in_seq) // self.batch_size
        else:
            return (sum(self.data_source.num_peds_in_seq) + self.batch_size - 1) // self.batch_size


def traj_collate_fn(data):
    r"""Collate function for the dataloader

    Args:
        data (list): list of tuples of (obs_seq, pred_seq, non_linear_ped, loss_mask, seq_start_end)

    Returns:
        obs_seq_list (torch.Tensor): (num_ped, obs_len, 2)
        pred_seq_list (torch.Tensor): (num_ped, pred_len, 2)
        non_linear_ped_list (torch.Tensor): (num_ped,)
        loss_mask_list (torch.Tensor): (num_ped, obs_len + pred_len)
        scene_mask (torch.Tensor): (num_ped, num_ped)
        seq_start_end (torch.Tensor): (num_ped, 2)
    """

    obs_seq_list, pred_seq_list, non_linear_ped_list, loss_mask_list, _, _ = zip(*data)

    _len = [len(seq) for seq in obs_seq_list]
    cum_start_idx = [0] + np.cumsum(_len).tolist()
    seq_start_end = [[start, end] for start, end in zip(cum_start_idx, cum_start_idx[1:])]
    seq_start_end = torch.LongTensor(seq_start_end)
    scene_mask = torch.zeros(sum(_len), sum(_len), dtype=torch.bool)
    for idx, (start, end) in enumerate(seq_start_end):
        scene_mask[start:end, start:end] = 1

    out = [torch.cat(obs_seq_list, dim=0), torch.cat(pred_seq_list, dim=0),
           torch.cat(non_linear_ped_list, dim=0), torch.cat(loss_mask_list, dim=0), scene_mask, seq_start_end]
    return tuple(out)


def get_dataloader(data_dir, phase, obs_len, pred_len, batch_size):
    r"""Get dataloader for a specific phase

    Args:
        data_dir (str): path to the dataset directory
        phase (str): phase of the data, one of 'train', 'val', 'test'
        obs_len (int): length of observed trajectory
        pred_len (int): length of predicted trajectory
        batch_size (int): batch size

    Returns:
        loader_phase (torch.utils.data.DataLoader): dataloader for the specific phase
    """

    assert phase in ['train', 'val', 'test']

    data_set = data_dir + '/' + phase + '/'
    shuffle = True if phase == 'train' else False
    drop_last = True if phase == 'train' else False

    dataset_phase = TrajectoryDataset(data_set, obs_len=obs_len, pred_len=pred_len)
    sampler_phase = None
    if batch_size > 1:
        sampler_phase = TrajBatchSampler(dataset_phase, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    loader_phase = DataLoader(dataset_phase, collate_fn=traj_collate_fn, batch_sampler=sampler_phase, pin_memory=True)
    return loader_phase


In [3]:
dataset_dir = "datasets/eth"
obs_len = 8
pred_len = 12
batch_size = 32

loader_train = get_dataloader(dataset_dir, 'train', obs_len, pred_len, batch_size=batch_size)
loader_val = get_dataloader(dataset_dir, 'val', obs_len, pred_len, batch_size=batch_size)
loader_test = get_dataloader(dataset_dir, 'test', obs_len, pred_len, batch_size=1)

In [4]:
batch = next(iter(loader_train))

print(len(batch))
print(batch[0].shape)
print(batch[1].shape)
print(batch[2].shape)
print(batch[3].shape)
print(batch[4].shape)
print(batch[5].shape)

# obs_seq_list (torch.Tensor): (num_ped, obs_len, 2)
# pred_seq_list (torch.Tensor): (num_ped, pred_len, 2)
# non_linear_ped_list (torch.Tensor): (num_ped,)
# loss_mask_list (torch.Tensor): (num_ped, obs_len + pred_len)
# scene_mask (torch.Tensor): (num_ped, num_ped)
# seq_start_end (torch.Tensor): (num_ped, 2)

6
torch.Size([32, 8, 2])
torch.Size([32, 12, 2])
torch.Size([32])
torch.Size([32, 20])
torch.Size([32, 32])
torch.Size([8, 2])


In [5]:
# Check the data scale
# print(batch[0][..., 0].max())
# print(batch[0][..., 0].min())
# print(batch[0][..., 1].max())
# print(batch[0][..., 1].min())

# Check the data scale for all the test set
x_min_list = []
x_max_list = []
y_min_list = []
y_max_list = []
for batch in loader_test:
    x_min_list.append(batch[0][..., 0].min())
    x_max_list.append(batch[0][..., 0].max())
    y_min_list.append(batch[0][..., 1].min())
    y_max_list.append(batch[0][..., 1].max())

    x_min_list.append(batch[1][..., 0].min())
    x_max_list.append(batch[1][..., 0].max())
    y_min_list.append(batch[1][..., 1].min())
    y_max_list.append(batch[1][..., 1].max())

x_min = min(x_min_list)
x_max = max(x_max_list)
y_min = min(y_min_list)
y_max = max(y_max_list)
print("X min: ", x_min)
print("X max: ", x_max)
print("Y min: ", y_min)
print("Y max: ", y_max)

X min:  tensor(-7.6900)
X max:  tensor(13.8900)
Y min:  tensor(-1.8100)
Y max:  tensor(12.6700)


In [6]:
def predict_trajectory(trajectory):
    """Predict the next future 12 frames using constant velocity
    Args:
        - trajectory [num_agents, traj_length, 2]: here the traj_length is 8;
    """
    current_pos = trajectory[:, -1, :]
    velocity = trajectory[:, -1, :] - trajectory[:, -2, :] # only use the last two frames
    predictions = []
    for t in range(1, 12+1): # 12 future frames
        current_pos = current_pos + velocity * 1 # dt
        predictions.append(current_pos)
    pred_trajectory = np.stack(predictions, axis=1)
    return pred_trajectory

batch = next(iter(loader_test))
print(batch[0].shape)

print(predict_trajectory(batch[0].numpy()).shape)

torch.Size([2, 8, 2])
(2, 12, 2)


In [7]:
# Evaluation function
def compute_batch_ade(pred, gt):
    r"""Compute ADE(average displacement error) scores for each pedestrian

    Args:
        pred (torch.Tensor): (num_samples, num_ped, seq_len, 2)
        gt (torch.Tensor): (1, num_ped, seq_len, 2) or (num_ped, seq_len, 2)

    Returns:
        ADEs (np.ndarray): (num_ped,)
    """

    temp = (pred - gt).norm(p=2, dim=-1)
    ADEs = temp.mean(dim=2).min(dim=0)[0]
    return ADEs.detach().cpu().numpy()


def compute_batch_fde(pred, gt):
    r"""Compute FDE(final displacement error) scores for each pedestrian

    Args:
        pred (torch.Tensor): (num_samples, num_ped, seq_len, 2)
        gt (torch.Tensor): (1, num_ped, seq_len, 2) or (num_ped, seq_len, 2)

    Returns:
        FDEs (np.ndarray): (num_ped,)
    """

    temp = (pred - gt).norm(p=2, dim=-1)
    FDEs = temp[:, :, -1].min(dim=0)[0]
    return FDEs.detach().cpu().numpy()


def compute_batch_tcc(pred, gt):
    r"""Compute TCC(temporal correlation coefficient) scores for each pedestrian

    Args:
        pred (torch.Tensor): (num_samples, num_ped, seq_len, 2)
        gt (torch.Tensor): (1, num_ped, seq_len, 2) or (num_ped, seq_len, 2)

    Returns:
        TCCs (np.ndarray): (num_ped,)
    """

    gt = gt.squeeze(dim=0) if gt.dim() == 4 else gt
    temp = (pred - gt).norm(p=2, dim=-1)
    pred_best = pred[temp[:, :, -1].argmin(dim=0), range(pred.size(1)), :, :]
    pred_gt_stack = torch.stack([pred_best, gt], dim=0)
    pred_gt_stack = pred_gt_stack.permute(3, 1, 0, 2)
    covariance = pred_gt_stack - pred_gt_stack.mean(dim=-1, keepdim=True)
    factor = 1 / (covariance.shape[-1] - 1)
    covariance = factor * covariance @ covariance.transpose(-1, -2)
    variance = covariance.diagonal(offset=0, dim1=-2, dim2=-1)
    stddev = variance.sqrt()
    corrcoef = covariance / stddev.unsqueeze(-1) / stddev.unsqueeze(-2)
    corrcoef = corrcoef.clamp(-1, 1)
    corrcoef[torch.isnan(corrcoef)] = 0
    TCCs = corrcoef[:, :, 0, 1].mean(dim=0)
    return TCCs.detach().cpu().numpy()


def compute_batch_col(pred, gt):
    r"""Compute COL(collision rate) scores for each pedestrian

    Args:
        pred (torch.Tensor): (num_samples, num_ped, seq_len, 2)
        gt (torch.Tensor): (1, num_ped, seq_len, 2) or (num_ped, seq_len, 2)

    Returns:
        COLs (np.ndarray): (num_ped,)
    """

    pred = pred.permute(0, 2, 1, 3)
    num_interp, thres = 4, 0.2
    pred_fp = pred[:, [0], :, :]
    pred_rel = pred[:, 1:] - pred[:, :-1]
    pred_rel_dense = pred_rel.div(num_interp).unsqueeze(dim=2).repeat_interleave(repeats=num_interp, dim=2).contiguous()
    pred_rel_dense = pred_rel_dense.reshape(pred.size(0), num_interp * (pred.size(1) - 1), pred.size(2), pred.size(3))
    pred_dense = torch.cat([pred_fp, pred_rel_dense], dim=1).cumsum(dim=1)
    col_mask = pred_dense[:, :3 * num_interp + 2].unsqueeze(dim=2).repeat_interleave(repeats=pred.size(2), dim=2)
    col_mask = (col_mask - col_mask.transpose(2, 3)).norm(p=2, dim=-1)
    col_mask = col_mask.add(torch.eye(n=pred.size(2), device=pred.device)[None, None, :, :]).min(dim=1)[0].lt(thres)
    COLs = col_mask.sum(dim=1).gt(0).type(pred.type()).mean(dim=0).mul(100)
    return COLs.detach().cpu().numpy()